In [1]:
!pip install torch
!pip install casadi

     |████████████████████████████████| 34.2MB 114kB/s 


In [1]:
import casadi as ca
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json
import itertools
import sys
sys.path.insert(0,"../test_system/quadrotor")
import params as par

In [ ]:
a = np.zeros(3,10)
a

TypeError: Cannot interpret '10' as a data type

In [3]:
def mix2motor(u_mix):
    '''from input of control to each motor duty'''
    map = np.linalg.inv(np.array([
        [-1,1,1,-1],
        [1,-1,1,-1],
        [1,1,-1,-1],
        [1,1,1,1]]))
    return ca.mtimes(map,u_mix)

def saturate(motor: ca.SX, len: int) -> ca.SX:
    ''' saturate the input motor voltage '''
    for i in range(4):
        v = motor[i]
        v = ca.if_else(v > 1, 1, ca.if_else(v < 0, 0, v))
        motor[i] = v
    return motor


In [39]:
u_mix = [-2,0,-2,0]
u_motor = saturate(mix2motor(u_mix), 4)                           # convert u_mix(angle input) to motor duty
u_motor

DM([0, 0, 0, 1])

In [31]:
mix2motor(np.array([[0],[0],[0],[12]]))

DM([3, 3, 3, 3])

In [16]:
def thrust(throttle, rho, r, V, kV, CT, Cm):
    """
    throttle: 0-1 controls voltage, []
    rho: density of air, [kg/m^3]
    r: radius of propeller, [m]
    V: voltage of battery [V]
    kV: motor kV constant, [rpm/V]
    """
    omega = throttle*V*kV*(2*ca.pi/60)
    q = 0.5*rho*(omega*r)**2
    s = ca.pi*r**2

    return CT*q*s,Cm*q*s

In [36]:
thrust(0, 1.224, 0.1, 11.1, 1550, 1.0e-2, 1e-4)

(0.0, 0.0)

In [33]:
np.array([[1,0,0],[0,0,1],[1,1,1]]).T

array([[1, 0, 1],
       [0, 0, 1],
       [0, 1, 1]])

In [3]:
map = np.linalg.inv(np.array([
        [-1,1,1,-1],
        [1,-1,1,-1],
        [1,1,-1,-1],
        [1,1,1,1]]))
map

array([[-0.25,  0.25,  0.25,  0.25],
       [ 0.25, -0.25,  0.25,  0.25],
       [ 0.25,  0.25, -0.25,  0.25],
       [-0.25, -0.25, -0.25,  0.25]])

In [10]:
1.0 == True

True

In [48]:
a = [1,0,1,1,1,0,0,0,1]
p = torch.FloatTensor([[1.0] if a_b else [0.0] for a_b in a])

pp = torch.FloatTensor()
for i in a:
    if i:
        pn = torch.FloatTensor([[1.0]])
    else:
        pn = torch.FloatTensor([[0.0]])
    pp = torch.cat((pp,pn), dim=0)
1-pp

tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.]])

In [22]:
a = np.array([])
a

array([], dtype=float64)

In [2]:
a = np.random.randint(0,10,(5,12))

In [45]:
a[4].sum()

43

In [46]:
a[2] = np.array([[]])

ValueError: could not broadcast input array from shape (0) into shape (12)

In [3]:
a

array([[3, 7, 0, 4, 0, 6, 6, 9, 3, 3, 1, 5],
       [8, 1, 9, 5, 6, 1, 3, 3, 4, 4, 0, 5],
       [6, 2, 6, 8, 6, 5, 7, 8, 0, 1, 1, 4],
       [5, 4, 7, 8, 0, 1, 1, 7, 2, 1, 8, 9],
       [7, 3, 3, 5, 6, 2, 9, 4, 5, 2, 6, 3]])

In [13]:
a[0,:4].mean()

3.5

In [14]:
a = np.zeros([3,32,100,12])
b = np.zeros([32,100,12])
c = np.zeros([32,100])
e = np.zeros([32,100,4])

In [16]:
a[0] = b
# a[1] = c
a[2] = e

ValueError: could not broadcast input array from shape (32,100,4) into shape (32,100,12)

In [11]:
a

array([  3,  32, 100,  12])

In [4]:
dic = {
    'a': np.array([1,2,3,4]),
    'b': np.array([[1,2,3],[4,5,6]]),
    'c': 4
}
np.save('flight_data.npy', dic)
# json.dump(dic, open('../flight_data.json', 'w'))

In [24]:
data = np.load('flight_data.npy',allow_pickle=True).item()
data['a'][:3]

array([1, 2, 3])

In [26]:
len(np.zeros(4))

4

In [168]:
action_roll = par.action_roll               # [V]
action_pitch = par.action_pitch
action_yaw = par.action_yaw
action_thrust = par.action_thrust

action_roll = np.array([action_roll,0,0,0])
action_pitch = np.array([0,action_pitch,0,0])
action_yaw = np.array([0,0,action_yaw,0])
action_thrust = np.array([0,0,0,action_thrust])
actions = np.vstack((action_roll, action_pitch, action_yaw, action_thrust, -action_roll, -action_pitch, -action_yaw, -action_thrust))
k = np.zeros(4)
dic_k = {}


for j in range(8):
    for i in range(len(list(itertools.combinations(actions, j+1)))):
        k = np.vstack((k,sum(list(itertools.combinations(actions, j+1))[i])))

k = np.unique(k,axis=0)

for q in range(len(k)):
    dic_k[q] = k[q]

dic_k

{0: array([-0.001, -0.001, -0.001, -0.01 ]),
 1: array([-0.001, -0.001, -0.001,  0.   ]),
 2: array([-0.001, -0.001, -0.001,  0.01 ]),
 3: array([-0.001, -0.001,  0.   , -0.01 ]),
 4: array([-0.001, -0.001,  0.   ,  0.   ]),
 5: array([-0.001, -0.001,  0.   ,  0.01 ]),
 6: array([-0.001, -0.001,  0.001, -0.01 ]),
 7: array([-0.001, -0.001,  0.001,  0.   ]),
 8: array([-0.001, -0.001,  0.001,  0.01 ]),
 9: array([-0.001,  0.   , -0.001, -0.01 ]),
 10: array([-0.001,  0.   , -0.001,  0.   ]),
 11: array([-0.001,  0.   , -0.001,  0.01 ]),
 12: array([-0.001,  0.   ,  0.   , -0.01 ]),
 13: array([-0.001,  0.   ,  0.   ,  0.   ]),
 14: array([-0.001,  0.   ,  0.   ,  0.01 ]),
 15: array([-0.001,  0.   ,  0.001, -0.01 ]),
 16: array([-0.001,  0.   ,  0.001,  0.   ]),
 17: array([-0.001,  0.   ,  0.001,  0.01 ]),
 18: array([-0.001,  0.001, -0.001, -0.01 ]),
 19: array([-0.001,  0.001, -0.001,  0.   ]),
 20: array([-0.001,  0.001, -0.001,  0.01 ]),
 21: array([-0.001,  0.001,  0.   , -0.01 ])

In [131]:
list(itertools.combinations(actions, 8))

[(array([1, 0, 0, 0]),
  array([0, 1, 0, 0]),
  array([0, 0, 1, 0]),
  array([0, 0, 0, 1]),
  array([-1,  0,  0,  0]),
  array([ 0, -1,  0,  0]),
  array([ 0,  0, -1,  0]),
  array([ 0,  0,  0, -1]))]

In [154]:
u = np.array([0.0,0.0,0.0,14])
u.shape[0]

4

In [4]:
o = np.array([[1,2,3],
              [4,5,6]])
np.reshape(o,-1)

array([1, 2, 3, 4, 5, 6])

In [45]:
u_vect = ca.SX.sym('u', 10,4)

# u_vect
for i in ca.vertsplit(u_vect,1):
    print(i)

[[u_0, u_10, u_20, u_30]]
[[u_1, u_11, u_21, u_31]]
[[u_2, u_12, u_22, u_32]]
[[u_3, u_13, u_23, u_33]]
[[u_4, u_14, u_24, u_34]]
[[u_5, u_15, u_25, u_35]]
[[u_6, u_16, u_26, u_36]]
[[u_7, u_17, u_27, u_37]]
[[u_8, u_18, u_28, u_38]]
[[u_9, u_19, u_29, u_39]]


In [30]:
ca.sx.sym.nz?

Object `ca.sx.sym.nz` not found.


In [3]:
a = np.array([1,2,3])
a2 = np.array([2,3,4])

np.linalg.norm(a2 - a)

1.7320508075688772

In [14]:
a = ca.DM([0,1,2.3])
b = np.zeros([2,3])
b[0] = a.T
round(b,4)

TypeError: type numpy.ndarray doesn't define __round__ method

In [5]:
a = np.array([[2]])
b = 2

ca.dot(a,b)

DM(4)

In [3]:
np.zeros((3,1))

array([[0.],
       [0.],
       [0.]])